In [1]:
# Download the data
import os
import urllib.request
datapath = os.path.join("datasets", "lifesat", "")
DOWNLOAD_ROOT = "https://raw.githubusercontent.com/ageron/handson-ml2/master/"
os.makedirs(datapath, exist_ok=True)
for filename in ("oecd_bli_2015.csv", "gdp_per_capita.csv"):
    print("Downloading", filename)
    url = DOWNLOAD_ROOT + "datasets/lifesat/" + filename
    urllib.request.urlretrieve(url, datapath + filename)

In [26]:
def prepare_country_stats(oecd_bli, gdp_per_capita):
    oecd_bli = oecd_bli[oecd_bli["INEQUALITY"] == "TOT"]
    oecd_bli = oecd_bli.pivot(index="Country", columns="Indicator", values="Value")
    gdp_per_capita.rename(columns={"2020": "GDP per capita"}, inplace=True)
    print(gdp_per_capita)
    gdp_per_capita.set_index("Country", inplace=True)
    full_country_stats = pd.merge(
        left=oecd_bli, right=gdp_per_capita, left_index=True, right_index=True
    )
    full_country_stats.sort_values(by="GDP per capita", inplace=True)
    remove_indices = [0, 1, 6, 8, 33, 34, 35]
    keep_indices = list(set(range(36)) - set(remove_indices))
    return full_country_stats[["GDP per capita", "Life satisfaction"]].iloc[
        keep_indices
    ]

In [30]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sklearn.linear_model
import seaborn as sns
from rich import print

sns.set_theme(style="darkgrid")

# data loading
oecd_bli = pd.read_csv("datasets/lifesat/oecd_bli_2022.csv", thousands=",")
gdp_per_capita = pd.read_csv(
    "datasets/lifesat/gdp_per_capita_2022.csv",
    # thousands=",",
    # delimiter="\t",
    # encoding="latin1",
    # na_values="n/a",
    # encoding='utf-8'
)

# preping data
country_stats = prepare_country_stats(oecd_bli, gdp_per_capita)
X = np.c_[country_stats["GDP per capita"]]
y = np.c_[country_stats["Life satisfaction"]]

# visualize the data
sns.relplot(kind="scatter", x="GDP per capita", y="Life satisfaction", data=country_stats);

# select a linear model
model = sklearn.linear_model.LinearRegression()

# Train the model
model.fit(X, y)

# make a prediction for Cyprus
X_new = [[22587]]  # Cyprus's GDP per capita
print(model.predict(X_new))

ÿþC  Unnamed: 1  Unnamed: 2  Unnamed: 3  Unnamed: 4  Unnamed: 5  \
0    NaN         NaN         NaN         NaN         NaN         NaN   
1    NaN         NaN         NaN         NaN         NaN         NaN   
2    NaN         NaN         NaN         NaN         NaN         NaN   
3    NaN         NaN         NaN         NaN         NaN         NaN   
4    NaN         NaN         NaN         NaN         NaN         NaN   
..   ...         ...         ...         ...         ...         ...   
194  NaN         NaN         NaN         NaN         NaN         NaN   
195  NaN         NaN         NaN         NaN         NaN         NaN   
196  NaN         NaN         NaN         NaN         NaN         NaN   
197  NaN         NaN         NaN         NaN         NaN         NaN   
198  NaN         NaN         NaN         NaN         NaN         NaN   

     Unnamed: 6  Unnamed: 7  Unnamed: 8  Unnamed: 9  ...  Unnamed: 28  \
0           NaN         NaN         NaN         NaN  ...          NaN   
1           NaN         NaN         NaN         NaN  ...          NaN   
2           NaN         NaN         NaN         NaN  ...          NaN   
3           NaN         NaN         NaN         NaN  ...          NaN   
4           NaN         NaN         NaN         NaN  ...          NaN   
..          ...         ...         ...         ...  ...          ...   
194         NaN         NaN         NaN         NaN  ...          NaN   
195         NaN         NaN         NaN         NaN  ...          NaN   
196         NaN         NaN         NaN         NaN  ...          NaN   
197         NaN         NaN         NaN         NaN  ...          NaN   
198         NaN         NaN         NaN         NaN  ...          NaN   

     Unnamed: 29  Unnamed: 30  Unnamed: 31  Unnamed: 32  Unnamed: 33  \
0            NaN          NaN          NaN          NaN          NaN   
1            NaN          NaN          NaN          NaN          NaN   
2            NaN          NaN          NaN          NaN          NaN   
3            NaN          NaN          NaN          NaN          NaN   
4            NaN          NaN          NaN          NaN          NaN   
..           ...          ...          ...          ...          ...   
194          NaN          NaN          NaN          NaN          NaN   
195          NaN          NaN          NaN          NaN          NaN   
196          NaN          NaN          NaN          NaN          NaN   
197          NaN          NaN          NaN          NaN          NaN   
198          NaN          NaN          NaN          NaN          NaN   

     Unnamed: 34  Unnamed: 35  Unnamed: 36  Unnamed: 37  
0            NaN          NaN          NaN          NaN  
1            NaN          NaN          NaN          NaN  
2            NaN          NaN          NaN          NaN  
3            NaN          NaN          NaN          NaN  
4            NaN          NaN          NaN          NaN  
..           ...          ...          ...          ...  
194          NaN          NaN          NaN          NaN  
195          NaN          NaN          NaN          NaN  
196          NaN          NaN          NaN          NaN  
197          NaN          NaN          NaN          NaN  
198          NaN          NaN          NaN          NaN  

[199 rows x 38 columns]

KeyError: "None of ['Country'] are in the columns"

In [ ]:
import sklearn.neighbors

model1 = sklearn.neighbors.KNeighborsRegressor(n_neighbors=3)

# Train the model
model1.fit(X, y)

# Make a prediction for Cyprus
print(model1.predict(X_new))